In [ ]:
import logging

from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mla.metrics.metrics import mean_squared_error
from mla.neuralnet import NeuralNet
from mla.neuralnet.constraints import MaxNorm
from mla.neuralnet.layers import Activation, Dense, Dropout
from mla.neuralnet.optimizers import Adadelta, Adam
from mla.neuralnet.parameters import Parameters
from mla.neuralnet.regularizers import L2
from mla.utils import one_hot
import pandas as pd
from mla.neuralnet.layers.recurrent import LSTM, RNN
logging.basicConfig(level=logging.DEBUG)


def classification():
    # Generate a random binary classification problem.
    dataset_path = "d:/JuniorAI/Benchmarking-ML-Algorithms-/MLAlgorithms/class_imbalance/class_imbalance/dataset_764_analcatdata_apnea3.csv"
    data = pd.read_csv(dataset_path)

    # Supondo que o dataset tenha colunas 'features' e 'target'
    X = data.iloc[:, :-1].values  # Todas as colunas menos a última como features
    y = data.iloc[:, -1].values   # Última coluna como target
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    rnn_layer = LSTM(128, return_sequences=False)
    y = one_hot(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1111)

    model = NeuralNet(
        layers=[
            Dense(256, Parameters(init="uniform", regularizers={"W": L2(0.05)})),
            Activation("relu"),
            Dropout(0.5),
            Dense(128, Parameters(init="normal", constraints={"W": MaxNorm()})),
            Activation("relu"),
            Dense(2),
            Activation("softmax"),
        ],
        loss="categorical_crossentropy",
        optimizer=Adadelta(),
        metric="accuracy",
        batch_size=64,
        max_epochs=25,
    )
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    auc_score = roc_auc_score(y_test[:, 0], predictions[:, 0])  # Calculate AUC
    print("Roc Score Area under the curve", auc_score)
    fpr, tpr, thresholds = roc_curve(y_test[:, 0], predictions[:, 0])

    # Plot FPR vs TPR
    plt.figure()
    plt.plot(fpr, tpr, label="ROC Curve (AUC = {:.2f})".format(auc_score))  # Use AUC score here
    plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.title("Receiver Operating Characteristic (ROC) Curve")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()


def regression():
    # Generate a random regression problem
    X, y = make_regression(n_samples=5000, n_features=25, n_informative=25, n_targets=1, random_state=100, noise=0.05)
    y *= 0.01
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1111)

    model = NeuralNet(
        layers=[
            Dense(64, Parameters(init="normal")),
            Activation("linear"),
            Dense(32, Parameters(init="normal")),
            Activation("linear"),
            Dense(1),
        ],
        loss="mse",
        optimizer=Adam(),
        metric="mse",
        batch_size=256,
        max_epochs=15,
    )
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print("regression mse", mean_squared_error(y_test, predictions.flatten()))



classification()

INFO:root:Total parameters: 34178
  0%|          | 0/6 [00:00<?, ?it/s]


TypeError: can't multiply sequence by non-int of type 'float'